# Predicting/Classifying Defects Using Model

Predicting/Classifying Defects Using Model

In [ ]:
import cv2
import joblib
import warnings
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import matplotlib.patches as patches
from skimage.feature import local_binary_pattern
from skimage.feature import graycomatrix, graycoprops

The code utilizes a trained model by accessing its .pkl file and classifies defect types for new images based on the texture features of the image

In [ ]:

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Load the trained Random Forest model
clf = joblib.load('random_forest_model.pkl')

# Define the minimum and maximum values for normalization
min_contrast = 0.0
max_contrast = 1000.0
min_energy = 0.0
max_energy = 1.0
min_entropy = 0.0
max_entropy = 10.0  
min_homogeneity = 0.0
max_homogeneity = 1.0
min_correlation = -1.0
max_correlation = 1.0
min_dissimilarity = 0.0  
max_dissimilarity = 100.0  
min_asm = 0.00001  
max_asm = 0.05  

# Load the new image
image_path = #provide the image to classify 

# Load the corresponding XML file
xml_path = #provide the corresponding Labels in .xml file 

# Load the image
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Load the XML file
tree = ET.parse(xml_path)
root = tree.getroot()

# Create empty lists to store bounding box coordinates and defect predictions
bbox_coords = []
defect_predictions = []

# Define the common LBP and GLCM parameters
lbp_params = [(1, 8), (2, 8), (3, 8)]
glcm_params = [(1, 0), (1, np.pi/4), (1, np.pi/2), (1, 3*np.pi/4)]

# Iterate through all objects in the XML file
for obj in root.findall('.//object'):
    xmin = int(obj.find('bndbox/xmin').text)
    ymin = int(obj.find('bndbox/ymin').text)
    xmax = int(obj.find('bndbox/xmax').text)
    ymax = int(obj.find('bndbox/ymax').text)

    # Crop the region of interest (ROI) from the image based on the bounding box
    roi = image[ymin:ymax, xmin:xmax]

    # Initialize LBP and GLCM features
    lbp_features = []
    glcm_features = []

    # Calculate GLCM features for the ROI
    for glcm_distance, glcm_angle in glcm_params:
        # Calculate the GLCM
        glcm = graycomatrix(roi, [glcm_distance], [glcm_angle], levels=256)

        # Normalize the GLCM
        glcm_normalized = glcm.astype(np.float32) / np.sum(glcm)

        # Calculate texture properties
        contrast = graycoprops(glcm_normalized, 'contrast')[0, 0]
        energy = graycoprops(glcm_normalized, 'energy')[0, 0]
        entropy = -np.sum(glcm_normalized * np.log(glcm_normalized + np.finfo(float).eps))
        homogeneity = graycoprops(glcm_normalized, 'homogeneity')[0, 0]
        correlation = graycoprops(glcm_normalized, 'correlation')[0, 0]
        dissimilarity = graycoprops(glcm_normalized, 'dissimilarity')[0, 0]
        ASM = graycoprops(glcm_normalized, 'ASM')[0, 0]

        # Normalize the texture characteristics
        normalized_contrast = (contrast - min_contrast) / (max_contrast - min_contrast)
        normalized_energy = (energy - min_energy) / (max_energy - min_energy)
        normalized_entropy = (entropy - min_entropy) / (max_entropy - min_entropy)
        normalized_homogeneity = (homogeneity - min_homogeneity) / (max_homogeneity - min_homogeneity)
        normalized_correlation = (correlation - min_correlation) / (max_correlation - min_correlation)
        normalized_dissimilarity = (dissimilarity - min_dissimilarity) / (max_dissimilarity - min_dissimilarity)
        normalized_ASM = (ASM - min_asm) / (max_asm- min_asm)

        # Append the normalized results to the features list
        glcm_features.extend([normalized_contrast, normalized_energy, normalized_entropy, normalized_homogeneity, normalized_correlation, normalized_dissimilarity, normalized_ASM])

    # Calculate LBP features for the ROI
    for lbp_radius, lbp_n_points in lbp_params:
        lbp_image = local_binary_pattern(roi, lbp_n_points, lbp_radius, method='uniform')
        lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, lbp_n_points + 3), range=(0, lbp_n_points + 2))
        lbp_hist = lbp_hist.astype(np.float32)
        lbp_hist /= (lbp_hist.sum() + 1e-6)
        lbp_features.extend(lbp_hist)

    # Combine GLCM and LBP features for the specific ROI
    features = glcm_features + lbp_features

    # Predict the defect type using the trained model
    prediction = clf.predict([features])[0]

    # Append the bounding box coordinates and the prediction
    bbox_coords.append((xmin, ymin, xmax, ymax))
    defect_predictions.append(prediction)

# Display the image with bounding boxes and defect predictions
fig, ax = plt.subplots(1)
ax.imshow(image, cmap='gray')

for i, coords in enumerate(bbox_coords):
    xmin, ymin, xmax, ymax = coords
    defect_type = defect_predictions[i]

    # Create a rectangle patch for the bounding box
    rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=1, edgecolor='green', facecolor='none')

    # Add the rectangle to the plot
    ax.add_patch(rect)

    # Add text with the defect prediction
    plt.text(xmin, ymin - 5, f'Predicted: {defect_type}', color='green')

plt.axis('off')
plt.show()